# Predicción de Ciberataques con AutoML

Este notebook implementa un proceso de AutoML para la predicción de ciberataques.

## 1. Importación de Librerías

In [ ]:
# Importación de librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Importación de PyCaret para AutoML
from pycaret.classification import *

# Configuración de visualización
plt.style.use('ggplot')
sns.set(style="whitegrid")
%matplotlib inline

## 2. Carga y Exploración de Datos

En esta sección, cargaremos un conjunto de datos de ejemplo para la predicción de ciberataques. Para un MVP, utilizaremos un dataset sintético.

In [ ]:
# Función para generar datos sintéticos de ciberataques
def generate_synthetic_data(n_samples=1000):
  """Genera datos sintéticos para simular características de ciberataques.
  
  Args:
    n_samples: Número de muestras a generar
    
  Returns:
    DataFrame con datos sintéticos
  """
  # Características simuladas de tráfico de red
  np.random.seed(42)
  
  # Generamos características que podrían ser relevantes para detectar ciberataques
  data = {
    'packet_rate': np.random.exponential(scale=100, size=n_samples),  # Tasa de paquetes por segundo
    'avg_packet_size': np.random.normal(loc=500, scale=150, size=n_samples),  # Tamaño promedio de paquetes
    'connection_duration': np.random.exponential(scale=30, size=n_samples),  # Duración de conexión en segundos
    'port_number': np.random.randint(1, 65536, size=n_samples),  # Número de puerto
    'protocol_type': np.random.choice(['TCP', 'UDP', 'ICMP'], size=n_samples, p=[0.7, 0.2, 0.1]),  # Tipo de protocolo
    'src_bytes': np.random.exponential(scale=1000, size=n_samples),  # Bytes enviados desde origen
    'dst_bytes': np.random.exponential(scale=2000, size=n_samples),  # Bytes enviados desde destino
    'login_attempts': np.random.poisson(lam=0.5, size=n_samples),  # Intentos de inicio de sesión
    'num_failed_logins': np.random.poisson(lam=0.2, size=n_samples),  # Número de inicios de sesión fallidos
    'num_compromised': np.random.poisson(lam=0.1, size=n_samples),  # Número de condiciones comprometidas
    'root_shell': np.random.binomial(1, 0.05, size=n_samples),  # Si se obtuvo shell de root
    'num_root': np.random.poisson(lam=0.1, size=n_samples),  # Número de accesos root
    'num_file_creations': np.random.poisson(lam=1, size=n_samples),  # Número de operaciones de creación de archivos
    'num_shells': np.random.poisson(lam=0.01, size=n_samples),  # Número de prompts de shell
    'num_access_files': np.random.poisson(lam=0.5, size=n_samples),  # Número de operaciones en archivos de control de acceso
    'count': np.random.poisson(lam=5, size=n_samples),  # Número de conexiones al mismo host
    'srv_count': np.random.poisson(lam=3, size=n_samples),  # Número de conexiones al mismo servicio
    'serror_rate': np.random.beta(0.5, 10, size=n_samples),  # Porcentaje de conexiones con errores SYN
    'srv_serror_rate': np.random.beta(0.5, 10, size=n_samples),  # Porcentaje de conexiones con errores SYN al mismo servicio
    'rerror_rate': np.random.beta(0.5, 10, size=n_samples),  # Porcentaje de conexiones con errores REJ
    'srv_rerror_rate': np.random.beta(0.5, 10, size=n_samples),  # Porcentaje de conexiones con errores REJ al mismo servicio
    'same_srv_rate': np.random.beta(5, 1, size=n_samples),  # Porcentaje de conexiones al mismo servicio
    'diff_srv_rate': np.random.beta(1, 5, size=n_samples),  # Porcentaje de conexiones a diferentes servicios
    'dst_host_count': np.random.poisson(lam=10, size=n_samples),  # Número de conexiones al mismo host destino
    'dst_host_srv_count': np.random.poisson(lam=8, size=n_samples)  # Número de conexiones al mismo servicio destino
  }
  
  # Creamos el DataFrame
  df = pd.DataFrame(data)
  
  # Convertimos protocol_type a variables dummy
  df = pd.get_dummies(df, columns=['protocol_type'], drop_first=True)
  
  # Generamos la variable objetivo (ataque o no ataque)
  # Usamos una combinación de características para determinar si es un ataque
  prob_attack = (0.01 + 
                 0.1 * (df['serror_rate'] > 0.2) + 
                 0.1 * (df['rerror_rate'] > 0.2) + 
                 0.1 * (df['num_failed_logins'] > 0) + 
                 0.2 * (df['num_compromised'] > 0) + 
                 0.2 * (df['root_shell'] > 0) + 
                 0.1 * (df['num_root'] > 0) + 
                 0.1 * (df['num_shells'] > 0) + 
                 0.05 * (df['packet_rate'] > 300))
  
  df['attack'] = np.random.binomial(1, prob_attack)
  
  # Añadimos algunos tipos de ataques para los casos positivos
  attack_types = ['DoS', 'Probe', 'R2L', 'U2R']
  df['attack_type'] = 'normal'
  attack_mask = df['attack'] == 1
  df.loc[attack_mask, 'attack_type'] = np.random.choice(attack_types, size=attack_mask.sum())
  
  return df

# Generamos los datos sintéticos
df = generate_synthetic_data(n_samples=5000)

# Mostramos las primeras filas del DataFrame
df.head()

In [ ]:
# Información básica del dataset
print(f"Dimensiones del dataset: {df.shape}")
print(f"\nDistribución de la variable objetivo (attack):\n{df['attack'].value_counts(normalize=True) * 100}")
print(f"\nDistribución de tipos de ataque:\n{df['attack_type'].value_counts()}")

# Estadísticas descriptivas
df.describe()

In [ ]:
# Visualización de la distribución de ataques
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
sns.countplot(x='attack', data=df)
plt.title('Distribución de Ataques')
plt.xlabel('Es Ataque (1=Sí, 0=No)')
plt.ylabel('Cantidad')

plt.subplot(1, 2, 2)
attack_counts = df['attack_type'].value_counts()
sns.barplot(x=attack_counts.index, y=attack_counts.values)
plt.title('Tipos de Ataques')
plt.xlabel('Tipo de Ataque')
plt.ylabel('Cantidad')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Análisis de correlación
plt.figure(figsize=(20, 16))
corr_matrix = df.drop(['attack_type'], axis=1).corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Matriz de Correlación de Características', fontsize=16)
plt.show()

# Correlación con la variable objetivo
corr_with_target = corr_matrix['attack'].sort_values(ascending=False)
print("\nCorrelación con la variable objetivo (attack):")
print(corr_with_target)

## 3. Preprocesamiento de Datos

In [ ]:
# Separamos características y variable objetivo
X = df.drop(['attack', 'attack_type'], axis=1)
y = df['attack']

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

print(f"Dimensiones de X_train: {X_train.shape}")
print(f"Dimensiones de X_test: {X_test.shape}")
print(f"Distribución de clases en entrenamiento:\n{y_train.value_counts(normalize=True) * 100}")
print(f"Distribución de clases en prueba:\n{y_test.value_counts(normalize=True) * 100}")

## 4. Proceso de AutoML con PyCaret

Utilizaremos PyCaret para automatizar el proceso de selección y optimización de modelos.

In [ ]:
# Preparamos los datos para PyCaret
# Combinamos X_train e y_train para el setup de PyCaret
train_data = X_train.copy()
train_data['attack'] = y_train

# Inicializamos el experimento de clasificación
clf_setup = setup(
  data=train_data,
  target='attack',
  train_size=0.7,  # 70% para entrenamiento, 30% para validación interna
  normalize=True,  # Normalización de características
  transformation=True,  # Transformación de características
  ignore_features=None,  # No ignoramos ninguna característica
  session_id=42,  # Para reproducibilidad
  silent=True,  # No mostrar mensajes de progreso
  verbose=True  # Mostrar resultados detallados
)

In [ ]:
# Comparamos todos los modelos disponibles
best_models = compare_models(
  fold=5,  # Validación cruzada con 5 folds
  sort='Accuracy',  # Ordenar por precisión
  n_select=5  # Seleccionar los 5 mejores modelos
)

In [ ]:
# Analizamos el mejor modelo
best_model = best_models[0] if isinstance(best_models, list) else best_models
print(f"Mejor modelo: {best_model}")

# Evaluamos el modelo
evaluate_model(best_model)

In [ ]:
# Ajustamos hiperparámetros del mejor modelo
tuned_model = tune_model(
  best_model,
  n_iter=50,  # Número de iteraciones para la búsqueda
  optimize='Accuracy',  # Métrica a optimizar
  search_library='optuna',  # Biblioteca de búsqueda
  choose_better=True  # Elegir el modelo con mejor rendimiento
)

In [ ]:
# Interpretación del modelo
interpret_model(tuned_model)

In [ ]:
# Predicciones en el conjunto de prueba
holdout_predictions = predict_model(tuned_model, data=X_test)

# Evaluación en el conjunto de prueba
print("\nRendimiento en el conjunto de prueba:")
print(f"Accuracy: {accuracy_score(y_test, holdout_predictions['prediction'])}")
print("\nInforme de clasificación:")
print(classification_report(y_test, holdout_predictions['prediction']))

# Matriz de confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, holdout_predictions['prediction'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('Matriz de Confusión')
plt.xlabel('Predicción')
plt.ylabel('Valor Real')
plt.show()

## 5. Ensamblaje de Modelos

Crearemos un modelo de ensamblaje para mejorar el rendimiento.

In [ ]:
# Creamos un modelo de ensamblaje (blending)
if isinstance(best_models, list) and len(best_models) > 1:
  blender = blend_models(
    estimator_list=best_models[:3],  # Usamos los 3 mejores modelos
    method='soft',  # Votación suave (probabilidades)
    optimize='Accuracy'  # Métrica a optimizar
  )
  
  # Evaluamos el modelo de ensamblaje
  print("\nRendimiento del modelo de ensamblaje:")
  evaluate_model(blender)
  
  # Predicciones con el modelo de ensamblaje
  ensemble_predictions = predict_model(blender, data=X_test)
  
  print(f"Accuracy del ensamblaje: {accuracy_score(y_test, ensemble_predictions['prediction'])}")
  print("\nInforme de clasificación del ensamblaje:")
  print(classification_report(y_test, ensemble_predictions['prediction']))
else:
  print("No hay suficientes modelos para crear un ensamblaje.")

## 6. Guardado del Modelo Final

In [ ]:
# Seleccionamos el modelo final (puede ser el modelo ajustado o el ensamblaje)
final_model = blender if 'blender' in locals() else tuned_model

# Finalizamos el modelo (entrenamiento en todos los datos)
final_model = finalize_model(final_model)

# Guardamos el modelo
save_model(final_model, '../models/cyberattack_predictor_model')

print("Modelo guardado exitosamente en '../models/cyberattack_predictor_model'")

## 7. Ejemplo de Uso del Modelo

In [ ]:
# Cargamos el modelo guardado
loaded_model = load_model('../models/cyberattack_predictor_model')

# Generamos algunos datos de ejemplo para predicción
example_data = generate_synthetic_data(n_samples=10).drop(['attack', 'attack_type'], axis=1)

# Realizamos predicciones
predictions = predict_model(loaded_model, data=example_data)

# Mostramos las predicciones
print("Predicciones para datos de ejemplo:")
print(predictions[['prediction_label', 'prediction_score']])

## 8. Conclusiones

En este notebook, hemos implementado un proceso completo de AutoML para la predicción de ciberataques:

1. Generamos datos sintéticos para simular características de ciberataques
2. Exploramos y visualizamos los datos
3. Utilizamos PyCaret para automatizar la selección y optimización de modelos
4. Evaluamos el rendimiento de los modelos
5. Creamos un modelo de ensamblaje para mejorar el rendimiento
6. Guardamos el modelo final para su uso en producción

Este MVP proporciona una base sólida para la predicción de ciberataques que puede ser mejorada con datos reales y características adicionales.